DATASET PREPROCESSING NOTEBOOK USING EG3D BY PHILLIP T. CHANANDA

CREDIT: NVIDIA LAB'S EG3D REPOSITORY
LINK: https://github.com/NVlabs/eg3d
PAPER: Efficient Geometry-aware {3D} Generative Adversarial Networks
AUTHORS: Eric R. Chan and Connor Z. Lin and Matthew A. Chan and Koki Nagano and Boxiao Pan and Shalini De Mello and Orazio Gallo and Leonidas Guibas and Jonathan Tremblay and Sameh Khamis and Tero Karras and Gordon Wetzstein



In [ ]:
# MOUNT YOUR GOOGLE DRIVE
# from google.colab import drive
# drive.mount("/content/drive/")

In [ ]:
# GIT CLONE THE REPOSITORY

In [ ]:
%cd "/content/drive/MyDrive/"
!ls

In [ ]:
!git clone https://github.com/NVlabs/eg3d.git
%cd eg3d

In [ ]:
# INSTALL CONDA
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!conda --version

In [ ]:
%cd /content/drive/MyDrive/eg3d/eg3d
!ls

In [ ]:
!mamba env update -f environment.yml

In [ ]:
%%shell
source activate eg3d

In [ ]:
# INSTALL DEPENDENCIES
!pip install numpy kornia dominate tensorflow tensorboard scipy opencv-python scikit-image ninja trimesh
!pip install mtcnn
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu119
!pip install torch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0 --index-url https://download.pytorch.org/whl/cu121

In [ ]:
# INSTALL NVIDIFFRAST
%cd /content/drive/MyDrive/eg3d/dataset_preprocessing/ffhq/Deep3DFaceRecon_pytorch/nvdiffrast
!pip install .

In [ ]:
%cd /content/drive/MyDrive/eg3d/dataset_preprocessing/ffhq/Deep3DFaceRecon_pytorch

In [ ]:
%cd /content/drive/MyDrive/eg3d/dataset_preprocessing/ffhq

In [ ]:
# INITIALIZE PATHS
# change to name of your dataset
dataset_name = "all"
dataset_path = "/content/drive/MyDrive/eg3d/dataset_preprocessing/ffhq/Deep3DFaceRecon_pytorch/datasets/"+dataset_name
print(dataset_path)

In [ ]:
# RUN MTCNN NEEDED FOR Deep3DFaceRecon
!python batch_mtcnn.py --in_root {dataset_path}

In [ ]:
# !pip install --upgrade torch torchvision

In [ ]:
%cd /content/drive/MyDrive/eg3d/dataset_preprocessing/ffhq/Deep3DFaceRecon_pytorch
!python test.py --img_folder={dataset_path} --gpu_ids=0 --name=pretrained --epoch=20

In [ ]:
# CROP THE IMAGES
%cd /content/drive/MyDrive/eg3d/dataset_preprocessing/ffhq
!python crop_images_in_the_wild.py --indir={dataset_path}

In [ ]:
# CONVERT POSE TO NEW FORMAT
%cd /content/drive/MyDrive/eg3d/dataset_preprocessing/ffhq/
!python 3dface2idr_mat.py --in_root /content/drive/MyDrive/eg3d/dataset_preprocessing/ffhq/Deep3DFaceRecon_pytorch/checkpoints/pretrained/results/{dataset_name}/epoch_20_000000 --out_path /content/drive/MyDrive/eg3d/dataset_preprocessing/ffhq/Deep3DFaceRecon_pytorch/datasets/{dataset_name}/crop/cameras.json

In [ ]:
# PREPROCESS FACE CAMERAS
!python preprocess_face_cameras.py --source /content/drive/MyDrive/eg3d/dataset_preprocessing/ffhq/Deep3DFaceRecon_pytorch/datasets/{dataset_name}/crop --dest /content/drive/MyDrive/eg3d/dataset_preprocessing/ffhq/Deep3DFaceRecon_pytorch/datasets/{dataset_name}/preprocessed_cameras --mode orig

In [ ]:
# CONCATENATE CAMERA_POSE
cameras_path = dataset_path+ "/crop/cameras.json"
print(cameras_path)
!python concate_camera_poses.py --cameras_path={cameras_path} --save_to={dataset_path}

In [ ]:
# MAKING SURE THE CAMERA_POSE FILES CORRESPONDS TO THE FINAL_CROPS FILES
import os
import cv2
import numpy as np
# SET YOUR DESIRED FINAL DATASET PATH
destination_folder = "/content/drive/MyDrive/3dSwap/datasets/"
cameras = dataset_path+"/camera_pose/"
crops = dataset_path+"/crop/"
final_crops = "final_crops"
camera_poses = "camera_pose"

# CHECK IF CAMERA_POSE HAVE MATCHING FINAL_CROPS
for camera in os.listdir(cameras):
    new_camera = camera[:-4]
    for crop in os.listdir(crops):
        new_crop = crop[:-4]
        new_path = destination_folder + dataset_name + "/"
        if os.path.exists(new_path)==False:
            os.mkdir(new_path)
        if new_camera == new_crop:
            image = cv2.imread(crops+crop)
            camera_npy_file = np.load(cameras+camera)
            if os.path.exists(new_path+final_crops) or os.path.exists(new_path+camera_poses):
                print("Path --- "+new_path+final_crops+"/"+crop)
                cv2.imwrite(new_path+final_crops+"/"+crop, image)
                np.save(new_path+camera_poses+"/"+camera, camera_npy_file)
            else:
                print("Path --- "+new_path+final_crops+"/"+crop)
                os.mkdir(new_path+final_crops)
                os.makedirs(new_path+camera_poses)
                cv2.imwrite(new_path+final_crops+"/"+crop, image)
                np.save(new_path+camera_poses+"/"+camera, camera_npy_file)

In [ ]:
# !pip install click
# %cd /content/drive/MyDrive/eg3d/eg3d
# !python dataset_tool.py --source {dataset_path} --dest {dataset_path}